In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt



In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

IMAGE_H = 1000
IMAGE_W = 1600

src = np.float32([[0, IMAGE_H], [1207, IMAGE_H], [0, 0], [IMAGE_W, 0]])
dst = np.float32([[569, IMAGE_H], [711, IMAGE_H], [0, 0], [IMAGE_W, 0]])
M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
Minv = cv2.getPerspectiveTransform(dst, src) # Inverse transformation

img = cv2.imread('nightway2.png') # Read the test img
img = img[450:(450+IMAGE_H), 0:IMAGE_W] # Apply np slicing for ROI crop
warped_img = cv2.warpPerspective(img, M, (IMAGE_W, IMAGE_H)) # Image warping
cv2.imwrite("test2.jpg", warped_img)
plt.imshow(cv2.cvtColor(warped_img, cv2.COLOR_BGR2RGB)) # Show results
plt.show()

TypeError: 'NoneType' object is not subscriptable

In [4]:
import numpy as np
import cv2 as cv
import moviepy.editor as mp

clip = mp.VideoFileClip("Road traffic video for object recognition")
clip_resized = clip.resize(height=360) # make the height 360px ( According to moviePy documenation The width is then computed so that the width/height ratio is conserved.)
clip_resized.write_videofile("movie_resized.mp4")


cap = cv.VideoCapture('movie_resized.mp4')
if cap.isOpened(): 
      ret,frame = cap.read()


# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))


# Create a mask image for drawing purposes
mask = np.zeros_like(frame)

while ret:
    ret, frame1 = cap.read()
    ret,frame2 = cap.read()
    old_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    frame_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks 
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame1 = cv.circle(frame1,(a,b),5,color[i].tolist(),-1)
        #frame1 = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
    img = cv.add(frame1,mask)

    cv.imshow('frame',img)
    if cv.waitKey(1) & 0xFF == 27:
         break
    frame1 = frame2
    ret,frame2= cap.read()
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv.destroyAllWindows()
cap.release()



    

ModuleNotFoundError: No module named 'moviepy'